# HDS5210-2020 Midterm

In the midterm, you're going to focus on using the programming skills that you've developed so far to build a calculator for the Apache II scoring system for ICU Mortality.  
* https://www.mdcalc.com/apache-ii-score#evidence
* https://reference.medscape.com/calculator/apache-ii-scoring-system

For the midterm, we'll be building a calculator for the Apache II score and then running that against a patient file that's available to you out on the internet.  This will be broken down into three main steps:
1. Create your JSON file to encapsulate all of the calculation rules for Apache II
2. Create functions to calculate the Apache II score using your JSON configuration
3. Create a function to loop over the patients in a file on the internet and calculate Apach II scores for all of them



---

## Part 1: Creating a JSON Rules File

Look at the rules for the Apache II scoring system on the pages above.  The first step in the midterm is to use those rules and create a JSON configuration file as described in the 2019 midterm video.  I've provided a starter file named `apache.json` to get you started.

Inside that file, you'll find placeholders for all of the measures that go into the Apache II scoring model:
* Organ Failure History
* Age
* Temperature
* [pH](https://en.wikipedia.org/wiki/PH)
* Heart rate
* Respiratory rate
* [Sodium](https://www.mayoclinic.org/diseases-conditions/hyponatremia/symptoms-causes/syc-20373711)
* [Potassium](https://www.emedicinehealth.com/hyperkalemia/article_em.htm)
* [Creatinine](https://www.medicalnewstoday.com/articles/322380)
* [Hematocrit](https://labtestsonline.org/tests/hematocrit)
* White Blood Count
* [FiO2](https://www.ausmed.com/cpd/articles/oxygen-flow-rate-and-fio2)
* [PaO2](https://www.verywellhealth.com/partial-pressure-of-oyxgen-pa02-914920)
* [A-a gradient](https://www.ncbi.nlm.nih.gov/books/NBK545153/)


You may need to create a sort of nested set of rules in some cases.  For instance, the rule for Creatinine says to use certain ranges and points in the case of Acute Renal Failure and a different set of points for Chronic Renal Failure.

Similarly, the rule for FiO2 says to use PaO2 to calculate scores if the FiO2 is <50, and to use A-a Gradient if the PaO2 is >50.

When you've created your `apache.json` file, make sure it's in the same directory as this notebook.

### Testing your JSON

The assert() functions below should all run just fine.  If you want to change the names of any of the keys in the JSON I provided you, you may, but you'll also need to update this test code so that it doesn't fail.  Remember, your notebook should be able to run end-to-end before you submit it.

In [1]:
import json

with open('apache.json') as f:
    rules = json.load(f)

assert('Organ Failure History' in rules.keys())
assert('Age' in rules.keys())
assert('Temperature' in rules.keys())
assert('pH' in rules.keys())
assert('Heart Rate' in rules.keys())
assert('Respiratory Rate' in rules.keys())
assert('Sodium' in rules.keys())
assert('Potassium' in rules.keys())
assert('Creatinine' in rules.keys())
assert('Hematocrit' in rules.keys())
assert('White Blood Count' in rules.keys())
assert('FiO2' in rules.keys())

---

## Part 2: Functions to evaluate rules

Write a series of functions, enough to satisfy all of the main criteria that we're using to calculate the Apache II score.  That list is the same as the assert statements above.

* Each of your functions should be well documented.
* Each function should have "config_file" as one of it's parameters.
* Each function should return a numerical score value.
* Similar to what we discussed in the review, if you can generalize some rules, do so.  You should **NOT** end up with one function for each input variable.  If you did that, you'd have a lot of repetative code.

The Glasgow Coma Scale is simply a 1-to-1 score translation.  Simply add the Glasgow Coma Scale value.  So, you don't need to write a function for this. [Glasgow Coma Scale](https://www.cdc.gov/masstrauma/resources/gcs.pdf)

**CORRECTION ADDED 2/29** - The Glasgow Coma Scale points should be calculated as `1 - Glasgow Coma Scale` rather than what I just stated above.  My preference would be that you do the calculation correctly, as per MDCalc, and then use the **corrected** scores files to compare against as noted in Part 4.

## #2A Organ Failure History

In [2]:
import json

def organ_history_score(history, config_file):
    """ str, str -> int
    history_list: A list of strings that include various organ failure history this subject DOES have
    config_file: The name of a configuration file to use for scoring
    
    The function should return a score for this subject based on the organ failure history in the history_list
    and the scoring rules in the config_file.  For instance, if the subject had Nonoperative,
    the history score would be 2, and the function should return 7.
    """
    
    config = json.load(open(config_file))
    organ_history_scores = config.get("Organ Failure History")
    
    if history in  organ_history_scores:
        history_score = organ_history_scores.get(history)

    return history_score


In [3]:
CONFIG_FILE = "apache.json"

In [4]:
assert( organ_history_score('Emergency', CONFIG_FILE) == 5)

## #2B Age

In [5]:
import json

def age_score(age, config_file):
    """ int, str -> int
    age: The age of the subject is an integer value
    config_file: The name of a configuration file to use for scoring
    
    The function should return a score for this subject based on where their actual age in the scoring rules.
    """
    config = json.load(open(config_file))
    age_rules = config.get("Age")
    
    for rule in age_rules:
        if age >= rule.get('min') and age < rule.get('max'):
            age_score = rule.get('points')
            
    return age_score

In [6]:
assert( age_score(30, CONFIG_FILE) == 0)

## #2C Temperature

In [7]:
import json

def temperature_score(temp, config_file):
    """ int, str -> int
    temp: The Temperature of the subject is an integer value
    config_file: The name of a configuration file to use for scoring 
    
    The function should return a score for this subject based on where their actual temperature
    in the scoring rules.
    """
    config = json.load(open(config_file))
    temp_rules = config.get("Temperature")
    
    for rule in temp_rules:
        if temp >= rule.get('min') and temp < rule.get('max'):
            temperature_score = rule.get('points')
            
    return temperature_score

In [8]:
assert( temperature_score(37, CONFIG_FILE) == 0)

## #2D pH

In [9]:
import json

def ph_score(ph, config_file):
    """ int, str -> int
    age: The ph of the subject is an integer value
    config_file: The name of a configuration file to use for scoring
    
    The function should return a score for this subject based on where their actual pH in 
    the scoring rules.
    """
    config = json.load(open(config_file))
    ph_rules = config.get("pH")
    
    for rule in ph_rules:
        if ph >= rule.get('min') and ph < rule.get('max'):
            ph_score = rule.get('points')
            
    return ph_score

In [10]:
assert( ph_score(6.5, CONFIG_FILE) == 4)

## #2E Heart Rate

In [11]:
import json

def hr_score(hr, config_file):
    """ int, str -> int
    age: The heart rate of the subject is an integer value
    config_file: The name of a configuration file to use for scoring
    
    The function should return a score for this subject based on where their actual heart
    rate in the scoring rules.
    """
    config = json.load(open(config_file))
    hr_rules = config.get("Heart Rate")
    
    for rule in hr_rules:
        if hr >= rule.get('min') and hr < rule.get('max'):
            hr_score = rule.get('points')
            
    return hr_score

In [12]:
assert( hr_score(50, CONFIG_FILE) == 3)

## #2F Respiratory Rate

In [13]:
import json

def rr_score(rr, config_file):
    """ int, str -> int
    age: The Respiratory Rate of the subject is an integer value
    config_file: The name of a configuration file to use for scoring
    
    The function should return a score for this subject based on where their actual 
    Respiratory Rate in the scoring rules.
    """
    config = json.load(open(config_file))
    rr_rules = config.get("Respiratory Rate")
    
    for rule in rr_rules:
        if rr >= rule.get('min') and rr < rule.get('max'):
            rr_score = rule.get('points')
            
    return rr_score

In [14]:
assert( rr_score(30, CONFIG_FILE) == 1)

## #2G Sodium

In [15]:
import json

def sodium_score(sodium, config_file):
    """ int, str -> int
    age: The Sodium of the subject is an integer value
    config_file: The name of a configuration file to use for scoring 
    
    The function should return a score for this subject based on where their actual 
    Sodium in the scoring rules.
    """
    config = json.load(open(config_file))
    sodium_rules = config.get("Sodium")
    
    for rule in sodium_rules:
        if sodium >= rule.get('min') and sodium < rule.get('max'):
            sodium_score = rule.get('points')
            
    return sodium_score

In [16]:
assert( sodium_score(100, CONFIG_FILE) == 4)

## #2H Potassium

In [17]:
import json

def potassium_score(potassium, config_file):
    """ float, str -> int
    age: The potassium of the subject is an float value
    config_file: The name of a configuration file to use for scoring
    
    The function should return a score for this subject based on where their actual 
    Respiratory Rate in the scoring rules.
    """
    config = json.load(open(config_file))
    potassium_rules = config.get("Potassium")
    
    for rule in potassium_rules:
        if potassium >= rule.get('min') and potassium < rule.get('max'):
            potassium_score = rule.get('points')
            
    return potassium_score

In [18]:
assert( potassium_score(2.8, CONFIG_FILE) == 2)

## #2I Creatinine

In [19]:
import json

def creatinine_score(failure_type, creatinine, config_file):
    """ str, int, str -> int
    age: The Hematocrit of the subject is an integer value
    config_file: The name of a configuration file to use for scoring
    
    The function should return a score for this subject based on where their actual 
    Hematocrit in the scoring rules.
    """
    config = json.load(open(config_file))
    failure_rules = config.get("Creatinine")
    acute_rules = failure_rules.get("Acute Renal Failure")
    chronic_rules = failure_rules.get("Chronic Renal Failure")
    if failure_type == "Acute Renal Failure":
        for rule1 in acute_rules:
            if creatinine >= rule1.get('min') and creatinine < rule1.get('max'):
                creatinine_score = rule1.get('points')
    if failure_type == "Chronic Renal Failure":
        for rule2 in chronic_rules:
            if creatinine >= rule2.get('min') and creatinine < rule2.get('max'):
                creatinine_score = rule2.get('points')
            
    return creatinine_score

In [20]:
assert(creatinine_score("Acute Renal Failure", 3, CONFIG_FILE) == 6)

## #2J Hematocrit

In [21]:
import json

def hematocrit_score(hema, config_file):
    """ int, str -> int
    age: The Hematocrit of the subject is an integer value
    config_file: The name of a configuration file to use for scoring
    
    The function should return a score for this subject based on where their actual 
    Hematocrit in the scoring rules.
    """
    config = json.load(open(config_file))
    hema_rules = config.get("Hematocrit")
    
    for rule in hema_rules:
        if hema >= rule.get('min') and hema < rule.get('max'):
            hema_score = rule.get('points')
            
    return hema_score

In [22]:
assert( hematocrit_score(25, CONFIG_FILE) == 2)

## #2K White Blood Count

In [23]:
import json

def wbc_score(wbc, config_file):
    """ int, str -> int
    age: The White Blood Count of the subject is an integer value
    config_file: The name of a configuration file to use for scoring
    
    The function should return a score for this subject based on where their actual 
    White Blood Count in the scoring rules.
    """
    config = json.load(open(config_file))
    wbc_rules = config.get("White Blood Count")
    
    for rule in wbc_rules:
        if wbc >= rule.get('min') and wbc < rule.get('max'):
            wbc_score = rule.get('points')
            
    return wbc_score

In [24]:
assert( wbc_score(2, CONFIG_FILE) == 2)

## #2L FiO2

In [25]:
import json

def fio2_score(fio2, aag, pao2, config_file):
    """ int, int, int, str -> int
    fio2: The FiO2 pecentage is an integer value
    aag: The A-a gradient is an integer value
    pao2: The PaO2 is an integer value
    config_file: The name of a configuration file to use for scoring
    
    The function should return a score for this subject based on where their actual FiO2 percentage,
    A-a gradient and PaO2 in the scoring rules.
    """
    config = json.load(open(config_file))
    fio2_rules = config.get("FiO2")
    aag_rules = fio2_rules.get("A-a Gradient")
    pao2_rules = fio2_rules.get("PaO2")
    fio2_score = 0
    if fio2 >= 0 and fio2 < 50:
         for rule1 in pao2_rules:
            if pao2 >= rule1.get('min') and pao2 < rule1.get('max'):
                fio2_score = rule1.get('points')
    if fio2 > 50 and fio2 <= 100:
        for rule2 in aag_rules:
            if aag >= rule2.get('min') and aag < rule2.get('max'):
                fio2_score = rule2.get('points')
    
    return fio2_score

In [26]:
assert(fio2_score(70, 268, 65, CONFIG_FILE) == 2)

In [27]:
fio2_score(70, 300, 65, CONFIG_FILE)

2

### Testing you Functions

Write enough test cases to verify that your functions work for evaulating all of the scoring inputs.  Have at least 3 test cases for each input.

These tests can be written the same as the assertions we've use in previous assignments.  For example, if you a function for `temperature_score` then you write a test case like:

```
assert( temperature_score(37) == 0 )
```

In [28]:
assert( organ_history_score('Emergency', CONFIG_FILE) == 5)
assert( age_score(30, CONFIG_FILE) == 0)
assert( temperature_score(37, CONFIG_FILE) == 0)
assert( ph_score(6.5, CONFIG_FILE) == 4)
assert( hr_score(50, CONFIG_FILE) == 3)
assert( rr_score(30, CONFIG_FILE) == 1)
assert( sodium_score(100, CONFIG_FILE) == 4)
assert( potassium_score(2.8, CONFIG_FILE) == 2)
assert(creatinine_score("Acute Renal Failure", 3, CONFIG_FILE) == 6)
assert( hematocrit_score(25, CONFIG_FILE) == 2)
assert( wbc_score(2, CONFIG_FILE) == 2)
assert(fio2_score(70, 300, 65, CONFIG_FILE) == 2)

---

## Part 3: Put it all together

Create a new function called `apache_score()` that takes all of the necessary inputs and returns the final Apache II score.  Use any variable names that you want.  For clarity and organization, my recommendation is to create them in the same order as they're documented in the website.

1. Organ Failure History
2. Age
3. Temperature
4. pH 
5. Heart rate
6. Respiratory rate
7. Sodium
8. Potassium
9. Creatinine
10. Acute renal failure
11. Hematocrit
12. White Blood Count
13. Glasgow Coma Scale
14. FiO2
15. PaO2
16. A-a gradient


In [29]:
def apache_score(history,age,temp,ph,hr,rr,sodium,potassium,failure_type,creatinine,hema,wbc,fio2,aag,pao2,gcs):
    
    # This function is a sumation of all the score.
    total_score = 0
    config_file = "apache.json"
    total_score += organ_history_score(history, config_file)
    total_score += age_score(age, config_file)
    total_score += temperature_score(temp, config_file)
    total_score += ph_score(ph, config_file)
    total_score += hr_score(hr, config_file)
    total_score += rr_score(rr, config_file)
    total_score += sodium_score(sodium, config_file)
    total_score += potassium_score(potassium, config_file)
    total_score += creatinine_score(failure_type, creatinine, config_file)
    total_score += hematocrit_score(hema, config_file)
    total_score += wbc_score(wbc, config_file)
    total_score += fio2_score(fio2, aag, pao2, config_file)
    total_score += (15-gcs)
    
    return total_score

### Testing your Function

Write a few test cases to make sure that your code functions correctly.  In the last step, you'll have LOTS of test cases run through, but you should do some of your before moving on.

In [30]:
apache_score('Emergency',6,27,7.3,174,10,170,7.8,"Chronic Renal Failure", 3,28,2,61, 458, 59,13) 

34

---

## Part 4: Accessing and processing the patient file

Fill out the simple function below to retrieve the patient data as a CSV file from any given URL and return a list of all of the Apache II scores based on the data you find for those patients.
* The patient file will be a CSV
* It will have column headers that match the labels shown above
* The columns will not necessarily appear in the order shown above
* You should output only the Apache II scores, not any other information
* Your output should be a list in the same order as the input rows

### Testing your Function

The URL for the test data is: https://hds5210-2020.s3.amazonaws.com/TestPatients.csv


You can verify your results by comparing them against this data: https://hds5210-2020.s3.amazonaws.com/Scores.csv

**CORRECTION ADDED 3/29** - If you calculated the Glasgow Coma Scale points as per the actual instructions in MDCalc, then please use this set of corrected scores to compare your results with: https://hds5210-2020.s3.amazonaws.com/Scores_corrected.csv


In [31]:
import csv
with open('TestPatients.csv') as f:
    patients = csv.reader(f)
    next(patients,None)
    my_scores = []
    for patient in patients:
        history = patient[1]
        age = int(patient[2])
        temp = int(patient[3])
        ph = float(patient[4])
        hr = int(patient[5])
        rr = int(patient[6])
        sodium = int(patient[7])
        potassium = float(patient[8])
        creatinine = float(patient[9])
        failure_type = patient[10]
        hema = int(patient[11])
        wbc = int(patient[12])
        gcs = int(patient[13])
        fio2 = int(patient[14])
        pao2 = int(patient[15])
        aag = int(patient[16])
        total_score = float(apache_score(history,age,temp,ph,hr,rr,sodium,potassium,failure_type,creatinine,hema,wbc,fio2,aag,pao2,gcs))
        my_scores.append(total_score)
print(my_scores)

# (my_scores) is the list of scores based on my approach.

[35.0, 31.0, 47.0, 34.0, 44.0, 35.0, 31.0, 49.0, 40.0, 48.0, 42.0, 43.0, 32.0, 41.0, 42.0, 49.0, 37.0, 37.0, 38.0, 43.0, 41.0, 31.0, 38.0, 30.0, 41.0, 41.0, 34.0, 46.0, 40.0, 47.0, 36.0, 43.0, 41.0, 46.0, 44.0, 40.0, 39.0, 37.0, 41.0, 30.0, 46.0, 30.0, 41.0, 44.0, 35.0, 36.0, 40.0, 40.0, 30.0, 50.0, 52.0, 43.0, 46.0, 34.0, 33.0, 42.0, 41.0, 31.0, 46.0, 46.0, 34.0, 36.0, 33.0, 38.0, 26.0, 29.0, 46.0, 25.0, 40.0, 38.0, 39.0, 34.0, 39.0, 53.0, 44.0, 49.0, 37.0, 36.0, 36.0, 51.0, 33.0, 36.0, 43.0, 41.0, 24.0, 50.0, 29.0, 40.0, 36.0, 50.0, 29.0, 37.0, 34.0, 45.0, 34.0, 40.0, 37.0, 37.0, 47.0, 31.0, 33.0, 50.0, 28.0, 37.0, 33.0, 44.0, 40.0, 38.0, 40.0, 31.0, 44.0, 37.0, 44.0, 47.0, 25.0, 34.0, 32.0, 41.0, 38.0, 34.0, 38.0, 39.0, 41.0, 34.0, 29.0, 44.0, 22.0, 40.0, 34.0, 43.0, 40.0, 31.0, 35.0, 33.0, 27.0, 44.0, 52.0, 40.0, 53.0, 40.0, 40.0, 48.0, 31.0, 38.0, 49.0, 43.0, 35.0, 31.0, 36.0, 40.0, 34.0, 31.0, 30.0, 28.0, 38.0, 47.0, 32.0, 47.0, 31.0, 41.0, 40.0, 33.0, 35.0, 42.0, 49.0, 55.0, 51.

In [32]:
import csv
with open('Scores_corrected.csv') as f:
    corrected_scores = []
    scores = csv.reader(f)
    next(scores,None)
    for line in scores:
        corrected_scores.append(float(line[0]))
    print(corrected_scores)

# (corrected_scores) is the list of score read from "Scores_corrected.csv"

[35.0, 31.0, 47.0, 34.0, 44.0, 35.0, 31.0, 49.0, 40.0, 48.0, 42.0, 43.0, 32.0, 41.0, 42.0, 49.0, 37.0, 37.0, 38.0, 43.0, 41.0, 31.0, 38.0, 30.0, 41.0, 41.0, 34.0, 46.0, 40.0, 47.0, 36.0, 43.0, 41.0, 46.0, 44.0, 40.0, 39.0, 37.0, 41.0, 30.0, 46.0, 30.0, 41.0, 44.0, 35.0, 36.0, 40.0, 40.0, 30.0, 50.0, 52.0, 43.0, 46.0, 34.0, 33.0, 42.0, 41.0, 31.0, 46.0, 46.0, 34.0, 36.0, 33.0, 38.0, 26.0, 29.0, 46.0, 25.0, 40.0, 38.0, 39.0, 34.0, 39.0, 53.0, 44.0, 49.0, 37.0, 36.0, 36.0, 51.0, 33.0, 36.0, 43.0, 41.0, 24.0, 50.0, 29.0, 40.0, 36.0, 50.0, 29.0, 37.0, 34.0, 45.0, 34.0, 40.0, 37.0, 37.0, 47.0, 31.0, 33.0, 50.0, 28.0, 37.0, 33.0, 44.0, 40.0, 38.0, 40.0, 31.0, 44.0, 37.0, 44.0, 47.0, 25.0, 34.0, 32.0, 41.0, 38.0, 34.0, 38.0, 39.0, 41.0, 34.0, 29.0, 44.0, 22.0, 40.0, 34.0, 43.0, 40.0, 31.0, 35.0, 33.0, 27.0, 44.0, 52.0, 40.0, 53.0, 40.0, 40.0, 48.0, 31.0, 42.0, 49.0, 43.0, 35.0, 31.0, 36.0, 42.0, 34.0, 31.0, 30.0, 28.0, 38.0, 47.0, 32.0, 47.0, 31.0, 41.0, 40.0, 33.0, 35.0, 42.0, 49.0, 55.0, 51.

In [33]:
loc_diff = []
for loc in range(0,len(my_scores)):
    if my_scores[loc] != corrected_scores[loc]:
        loc_diff.append(loc)
print(loc_diff)
    
# It seems there is only five of them uncorrect? 
# I think I am using the right approach. The accuracy is about 98.5% (329/334).
# The (loc_diff) shows the location of the uncorrect ones in the list.

[143, 149, 174, 263, 266]


In [35]:
# End